# 🖼️ Introduction to Multimodal RAG

In this notebook, you'll learn how to index and retrieve images using Haystack. By the end, you'll be able to build a Retrieval-Augmented Generation (RAG) pipeline that can answer questions grounded in both images and text. This is useful when working with datasets like scientific papers, diagrams, or screenshots where meaning is spread across modalities.

This tutorial uses the following **new components** that enable image indexing:

- `SentenceTransformersDocumentImageEmbedder`: Embed image documents with CLIP-based models
- `ImageFileToDocument`: Convert image files into Haystack `Document`s
- `DocumentTypeRouter`: Route retrieved documents by mime type (e.g., image vs text)
- `DocumentToImageContent`: Convert image documents into `ImageContent` to be processed by our ChatGenerator

In this notebook, we'll introduce all these features, show an application using **image + text retrieval + multimodal generation**.

## Setup Development Environment

In [ ]:
!pip install --upgrade "haystack-experimental" pillow pypdfium2

In [1]:
import os
from getpass import getpass
from pprint import pp as print


if "OPENAI_API_KEY" not in os.environ:
  os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

Enter OpenAI API key: ········


## Introduction to Embedding Images

Let's compare the similarity between a text and two images.

In [ ]:
!wget "https://upload.wikimedia.org/wikipedia/commons/2/26/Pink_Lady_Apple_%284107712628%29.jpg" -O apple.jpg
!wget "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Cattle_tyrant_%28Machetornis_rixosa%29_on_Capybara.jpg/960px-Cattle_tyrant_%28Machetornis_rixosa%29_on_Capybara.jpg?download" -O capybara.jpg


In [10]:
from haystack_experimental.components.converters.image import ImageFileToDocument

image_file_converter = ImageFileToDocument()
image_docs = image_file_converter.run(sources=["apple.jpg", "capybara.jpg"])["documents"]
print(image_docs)

Next we load our embedders. It's important that we use the same CLIP model for both text and images.

In [8]:
from haystack.components.embedders.sentence_transformers_text_embedder import SentenceTransformersTextEmbedder
from haystack_experimental.components.embedders.image.sentence_transformers_doc_image_embedder import (
    SentenceTransformersDocumentImageEmbedder,
)

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/clip-ViT-L-14", progress_bar=False)
image_embedder = SentenceTransformersDocumentImageEmbedder(model="sentence-transformers/clip-ViT-L-14", progress_bar=False)

# Warm up the models to load them
text_embedder.warm_up()
image_embedder.warm_up()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [16]:
import torch
from sentence_transformers import util

query = "A red apple on a white background"
text_embedding = text_embedder.run(text=query)["embedding"]
image_docs_with_embeddings = image_embedder.run(image_docs)["documents"]

# Compare the similarities between the query and two image documents
for doc in image_docs_with_embeddings:
    similarity = util.cos_sim(torch.tensor(text_embedding), torch.tensor(doc.embedding))
    print(f"Similarity with {doc.meta['file_path'].split('/')[-1]}: {similarity.item():.2f}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Similarity with apple.jpg: 0.27'
'Similarity with capybara.jpg: 0.07'


As we can see the text is most similar to our Apple image!

## Building an Image & Text Indexing Pipeline

First let's also download a sample PDF file to see how we can retrieve over both text and image based documents

In [2]:
!wget


wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


### Manually Embed Text and Image Documents

In [24]:
# Imports
from haystack.components.converters.pypdf import PyPDFToDocument
from haystack.components.embedders.sentence_transformers_document_embedder import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack.components.writers.document_writer import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore

In [26]:
# Create our document store
doc_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

# Define our components
image_converter = ImageFileToDocument()
pdf_converter = PyPDFToDocument()
pdf_splitter = DocumentSplitter(split_by="page", split_length=1)
text_doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/clip-ViT-L-14")

text_doc_embedder.warm_up()

In [29]:
# Create our pdf + image documents
pdf_docs = pdf_converter.run(sources=["sample.pdf"])["documents"]
split_pdf_docs = pdf_splitter.run(documents=pdf_docs)["documents"]
image_docs = image_converter.run(sources=["apple.jpg"])["documents"]

# Embed our text
pdf_docs_with_embeddings = text_doc_embedder.run(split_pdf_docs)['documents']
img_docs_with_embeddings = image_embedder.run(image_docs)['documents']

2025-07-03T09:44:21.727630Z [warning  ] Could not read sample.pdf. Skipping it. Error: [Errno 2] No such file or directory: 'sample.pdf' error=FileNotFoundError(2, 'No such file or directory') lineno=203 module=haystack.components.converters.pypdf source=sample.pdf


Batches: 0it [00:00, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
# Write our documents to the document store
doc_store.write_documents(pdf_docs_with_embeddings + img_docs_with_embeddings, policy="overwrite")

1

### Create an Indexing Pipeline to Process All Files at Once

In [31]:
# Imports
from haystack.components.routers.file_type_router import FileTypeRouter
from haystack.components.joiners import DocumentJoiner

In [35]:
# Additional component definitions
file_type_router = FileTypeRouter(mime_types=["application/pdf", "image/jpeg"])
final_doc_joiner = DocumentJoiner(sort_by_score=False)

In [36]:
# Create the Indexing Pipeline
from haystack import Pipeline

indexing_pipe = Pipeline()
indexing_pipe.add_component("file_type_router", file_type_router)
indexing_pipe.add_component("pdf_converter", pdf_converter)
indexing_pipe.add_component("pdf_splitter", pdf_splitter)
indexing_pipe.add_component("image_converter", image_converter)
indexing_pipe.add_component("text_doc_embedder", text_doc_embedder)
indexing_pipe.add_component("image_doc_embedder", image_embedder)
indexing_pipe.add_component("final_doc_joiner", final_doc_joiner)
indexing_pipe.add_component("document_writer", document_writer)

indexing_pipe.connect("file_type_router.application/pdf", "pdf_converter.sources")
indexing_pipe.connect("pdf_converter.documents", "pdf_splitter.documents")
indexing_pipe.connect("pdf_splitter.documents", "text_doc_embedder.documents")
indexing_pipe.connect("file_type_router.image/jpeg", "image_converter.sources")
indexing_pipe.connect("image_converter.documents", "image_doc_embedder.documents")
indexing_pipe.connect("text_doc_embedder.documents", "final_doc_joiner.documents")
indexing_pipe.connect("image_doc_embedder.documents", "final_doc_joiner.documents")
indexing_pipe.connect("final_doc_joiner.documents", "document_writer.documents")

PipelineError: Component has already been added in another Pipeline. Components can't be shared between Pipelines. Create a new instance instead.

Visualize the Indexing pipeline

In [ ]:
indexing_pipe.draw()

Run the indexing pipeline

In [ ]:
indexing_result = indexing_pipe.run(
    data={"file_type_router": {"sources": ["sample.pdf", "apple.jpg"]}},
)

Inspect the documents

In [ ]:
indexed_documents = document_store.filter_documents()
print(f"Indexed {len(indexed_documents)} documents:")
print(indexed_documents)

## Retrieval – Searching Image + Text

In [ ]:
retriever = InMemoryEmbeddingRetriever(document_store=doc_store)
results = retriever.run(query="An image of an apple")['documents']

for doc in results:
    print(doc.content[:100], doc.content_type, doc.meta.get("file_path"))

## RAG with Image + Text

In [ ]:
prompt_template = """
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Answer this query: {{ query }}
"""

rag = Pipeline()
rag.add_component("retriever", InMemoryEmbeddingRetriever(document_store=doc_store, top_k=3))
rag.add_component("router", DocumentTypeRouter())
rag.add_component("img_convert", DocumentToImageContent(detail="low"))
rag.add_component("prompt", PromptBuilder(template=prompt_template))
rag.add_component("llm", OpenAIChatGenerator(model="gpt-4o-mini"))

rag.connect("retriever.documents", "router.documents")
rag.connect("router.image", "img_convert.documents")
rag.connect("retriever.documents", "prompt.documents")
rag.connect("img_convert.image_contents", "prompt.image_contents")
rag.connect("prompt.prompt", "llm.messages")

response = rag.run({"query": "What does the image of the apple show?"})
print(response["llm"]["replies"][0].text)

## What's next?

You can follow the progress of the Multimodal experiment in this [GitHub issue](https://github.com/deepset-ai/haystack/issues/8976).

(*Notebook by [Sebastian Husch Lee](https://github.com/sjrl)*)